#### Faiss
Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning.

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader=TextLoader("./data/speech.txt")
documents=loader.load()
text_splitter=CharacterTextSplitter(separator="\n", chunk_size=50,chunk_overlap=20)
docs=text_splitter.split_documents(documents)

Created a chunk of size 73, which is longer than the specified 50
Created a chunk of size 412, which is longer than the specified 50
Created a chunk of size 168, which is longer than the specified 50
Created a chunk of size 118, which is longer than the specified 50


In [2]:
docs

[Document(metadata={'source': './data/speech.txt'}, page_content='Hello!'),
 Document(metadata={'source': './data/speech.txt'}, page_content='My name is DAVEWORLD, and I am excited to be your tutor for this program.'),
 Document(metadata={'source': './data/speech.txt'}, page_content='I am a Data Scientist, Open-source Contributor, and Frontend Engineer with years of experience in the tech industry. Throughout my career, I’ve been privileged to build innovative products, solve real-world problems, and empower over 5,000 individuals to grow into better versions of themselves. My work and insights have also connected me with an audience of over 10,000 followers across social media platforms.'),
 Document(metadata={'source': './data/speech.txt'}, page_content='Looking ahead, my vision is to create groundbreaking projects and support millions—if not billions—of people in achieving their goals and thriving in the tech industry.'),
 Document(metadata={'source': './data/speech.txt'}, page_cont

In [4]:
embeddings = (OllamaEmbeddings(model="gemma2:2b")) # by defaults llamma2
database = FAISS.from_documents(docs, embeddings)

In [5]:
database

In [6]:
query = "WHat's the speaker future ambition?"
retrived_result = database.similarity_search(query)
print(retrived_result)

[Document(id='4ec1d08b-e438-41c0-9cbf-7a07324044b4', metadata={'source': './data/speech.txt'}, page_content='Looking ahead, my vision is to create groundbreaking projects and support millions—if not billions—of people in achieving their goals and thriving in the tech industry.'), Document(id='d497400c-27fa-46e6-9bce-d25782a8009a', metadata={'source': './data/speech.txt'}, page_content='In this program, I am here to guide you every step of the way, ensuring you have an unforgettable learning experience.'), Document(id='2a18d4c6-7942-4f8f-8002-49cee08ab4ac', metadata={'source': './data/speech.txt'}, page_content='I am a Data Scientist, Open-source Contributor, and Frontend Engineer with years of experience in the tech industry. Throughout my career, I’ve been privileged to build innovative products, solve real-world problems, and empower over 5,000 individuals to grow into better versions of themselves. My work and insights have also connected me with an audience of over 10,000 followers

#### As a Retriever
We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers

In [10]:
retrieval = database.as_retriever()
docs = retrieval.invoke(query)
docs[0].page_content

'Looking ahead, my vision is to create groundbreaking projects and support millions—if not billions—of people in achieving their goals and thriving in the tech industry.'

#### Similarity Search with score
There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better.

In [11]:
docs_and_score = database.similarity_search_with_score(query)
docs_and_score

[(Document(id='4ec1d08b-e438-41c0-9cbf-7a07324044b4', metadata={'source': './data/speech.txt'}, page_content='Looking ahead, my vision is to create groundbreaking projects and support millions—if not billions—of people in achieving their goals and thriving in the tech industry.'),
  4627.6006),
 (Document(id='d497400c-27fa-46e6-9bce-d25782a8009a', metadata={'source': './data/speech.txt'}, page_content='In this program, I am here to guide you every step of the way, ensuring you have an unforgettable learning experience.'),
  5418.37),
 (Document(id='2a18d4c6-7942-4f8f-8002-49cee08ab4ac', metadata={'source': './data/speech.txt'}, page_content='I am a Data Scientist, Open-source Contributor, and Frontend Engineer with years of experience in the tech industry. Throughout my career, I’ve been privileged to build innovative products, solve real-world problems, and empower over 5,000 individuals to grow into better versions of themselves. My work and insights have also connected me with an au

In [12]:
embedding_vector = embeddings.embed_query(query)
embedding_vector

[-0.6297181844711304,
 1.94210684299469,
 -1.4898511171340942,
 0.6643972992897034,
 -1.1895604133605957,
 -0.551268458366394,
 0.897462785243988,
 -0.9539811015129089,
 0.7110637426376343,
 1.0620583295822144,
 1.026173710823059,
 -0.8261721730232239,
 -0.6739504933357239,
 -1.416894555091858,
 0.3613138794898987,
 -3.6238253116607666,
 0.16175216436386108,
 0.7011933922767639,
 -4.665972709655762,
 0.5434863567352295,
 1.6604150533676147,
 -0.6790810227394104,
 -1.41616952419281,
 0.43560418486595154,
 -0.10380357503890991,
 0.04784208908677101,
 -0.4895695745944977,
 -1.5505367517471313,
 -1.039842963218689,
 1.2034982442855835,
 0.12905548512935638,
 -1.645477533340454,
 -0.16743314266204834,
 0.07186418026685715,
 0.42498087882995605,
 1.666306495666504,
 0.4298882484436035,
 0.09445460140705109,
 1.7989983558654785,
 -2.015619993209839,
 -0.2573263943195343,
 2.4922964572906494,
 0.6339044570922852,
 -0.3625108003616333,
 -3.1435229778289795,
 -1.8743040561676025,
 0.647823929786

In [14]:
docs_score = database.similarity_search_with_score(embedding_vector)

In [15]:
print(docs_score)

[(Document(id='2a18d4c6-7942-4f8f-8002-49cee08ab4ac', metadata={'source': './data/speech.txt'}, page_content='I am a Data Scientist, Open-source Contributor, and Frontend Engineer with years of experience in the tech industry. Throughout my career, I’ve been privileged to build innovative products, solve real-world problems, and empower over 5,000 individuals to grow into better versions of themselves. My work and insights have also connected me with an audience of over 10,000 followers across social media platforms.'), 14313.063), (Document(id='4ec1d08b-e438-41c0-9cbf-7a07324044b4', metadata={'source': './data/speech.txt'}, page_content='Looking ahead, my vision is to create groundbreaking projects and support millions—if not billions—of people in achieving their goals and thriving in the tech industry.'), 15009.113), (Document(id='d497400c-27fa-46e6-9bce-d25782a8009a', metadata={'source': './data/speech.txt'}, page_content='In this program, I am here to guide you every step of the wa

In [16]:
# saving and loading the database
database.save_local("faiss_db")

In [17]:
new_database = FAISS.load_local("faiss_db", embeddings, allow_dangerous_deserialization=True)

In [18]:
docs = new_database.similarity_search(query)

In [20]:
docs[0].page_content

'Looking ahead, my vision is to create groundbreaking projects and support millions—if not billions—of people in achieving their goals and thriving in the tech industry.'